# Imports

In [ ]:
import pandas as pd
from hbt_tools import utils_hbt as uh
from hbt_tools import plots as hp
from pathlib import Path
from pytz import timezone

# Functions

In [ ]:
def ts_filter_date(ts, date_from=None, date_to=None):
    ts = ts.copy()
    if (date_from is None and date_to is None):
        pass

    elif (date_to is None and date_from is not None):
        ts = ts[ts.index>=date_from]
    
    elif (date_to is not None and date_from is None):
        ts = ts[ts.index<=date_to]
    else:
        ts = ts[(ts.index>=date_from) & (ts.index<=date_to)]

    return ts
#-------------------------------------------------------------------------------

def sum_per_period(ts, periods, date_format, idx_col=0):
    res = pd.DataFrame(columns=['von', 'bis', 'summe'])
    tz = timezone('Europe/Zurich')

    res_list = []
    for period in periods:
        date_start = pd.to_datetime(period[0], format=date_format)
        date_start = tz.localize(date_start)

        date_end = pd.to_datetime(period[1], format=date_format)
        date_end = tz.localize(date_end)

        ts_period = ts[(ts.index>=date_start) & (ts.index<date_end)]
        df_temp = pd.DataFrame({'von': [period[0]], 'bis': [period[1]],
            'summe': [ts_period.iloc[:,idx_col].sum()]})
        
        res_list.append(df_temp)
        
    res = pd.concat(res_list)
    
    return res

# Parameters

In [ ]:
date_format = '%d.%m.%Y %H:%M'
date_from = pd.to_datetime('12.09.2023 00:00', format=date_format)

date_to = pd.to_datetime('06.12.2023 00:00', format=date_format)

save_dir = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.43 SABA Zugerland\05 Berechnungen Grundlagen\Datenanalyse\Plots")

# Regen

In [ ]:
id = 2063
rain = uh.mdb_getdata(id, date_from, date_to)

# Read Data

## Niveau Pumpensumpf

In [ ]:
id = 2899
niveau_pw = uh.mdb_getdata(id, date_from, date_to)

## Niveau Retentionsfilterbecken

In [ ]:
id = 2900
rfb1 = uh.mdb_getdata(id, date_from, date_to)

id = 2901
rfb2 = uh.mdb_getdata(id, date_from, date_to)

# Plots

## Niveau Pumpensumpf

In [ ]:
save_path = save_dir / 'Niveau-Pumpensumpf.html'
fig = hp.ply_2y(niveau_pw, rain, names=['Niveau Pumpensumpf', 'Regen Cham'], save_path=save_path,
    xlabel='Zeit', ylabels=['Niveau [cm]', 'Regen [mm/10min]', 'Niveau Pumpensumpf'])
fig

## Niveau Retentionsfilterbecken

In [ ]:
save_path = save_dir / 'Niveau-Retentionsfilterbecken.html'
fig = hp.ply_2y([rfb1, rfb2], rain, names=['Niveau Retentionsfilterbecken 1', 'Niveau Retentionsfilterbecken 2', 'Regen Cham'],
    save_path=save_path, xlabel='Zeit', ylabels=['Niveau [cm]', 'Regen [mm/10min]', 'Niveau Pumpensumpf'])
fig